In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    FFF_Train_Decile2 = pd.read_pickle("FFF_Train_Decile2.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile2.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile2, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 2')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

Epoch 96/100
3055/3055 [==============================] - 2s 639us/step - loss: 0.4994 - acc: 0.7735 - val_loss: 0.4835 - val_acc: 0.7775
Epoch 97/100
3055/3055 [==============================] - 2s 639us/step - loss: 0.5032 - acc: 0.7712 - val_loss: 0.4833 - val_acc: 0.7775
Epoch 98/100
3055/3055 [==============================] - 2s 644us/step - loss: 0.4971 - acc: 0.7699 - val_loss: 0.4830 - val_acc: 0.7801
Epoch 99/100
3055/3055 [==============================] - 2s 661us/step - loss: 0.4984 - acc: 0.7735 - val_loss: 0.4829 - val_acc: 0.7788
Epoch 100/100
3055/3055 [==============================] - 2s 651us/step - loss: 0.5019 - acc: 0.7732 - val_loss: 0.4831 - val_acc: 0.7762
Test accuracy: 0.7761780107832704
Train on 3055 samples, validate on 764 samples
Epoch 1/150
3055/3055 [==============================] - 3s 931us/step - loss: 0.6218 - acc: 0.6534 - val_loss: 0.5573 - val_acc: 0.7225
Epoch 2/150
3055/3055 [==============================] - 2s 681us/step - loss: 0.5468 - acc

3055/3055 [==============================] - 2s 658us/step - loss: 0.4818 - acc: 0.7820 - val_loss: 0.4843 - val_acc: 0.7840
Epoch 113/150
3055/3055 [==============================] - 2s 663us/step - loss: 0.4890 - acc: 0.7813 - val_loss: 0.4845 - val_acc: 0.7840
Epoch 114/150
3055/3055 [==============================] - 2s 673us/step - loss: 0.4861 - acc: 0.7849 - val_loss: 0.4844 - val_acc: 0.7840
Epoch 115/150
3055/3055 [==============================] - 2s 676us/step - loss: 0.4845 - acc: 0.7804 - val_loss: 0.4842 - val_acc: 0.7840
Epoch 116/150
3055/3055 [==============================] - 2s 666us/step - loss: 0.4823 - acc: 0.7859 - val_loss: 0.4842 - val_acc: 0.7840
Epoch 117/150
3055/3055 [==============================] - 2s 678us/step - loss: 0.4838 - acc: 0.7859 - val_loss: 0.4842 - val_acc: 0.7840
Epoch 118/150
3055/3055 [==============================] - 2s 669us/step - loss: 0.4860 - acc: 0.7827 - val_loss: 0.4841 - val_acc: 0.7840
Epoch 119/150
3055/3055 [================

3055/3055 [==============================] - ETA: 0s - loss: 0.4922 - acc: 0.778 - 2s 701us/step - loss: 0.4911 - acc: 0.7797 - val_loss: 0.4834 - val_acc: 0.7801
Epoch 79/150
3055/3055 [==============================] - 2s 654us/step - loss: 0.4869 - acc: 0.7807 - val_loss: 0.4833 - val_acc: 0.7801
Epoch 80/150
3055/3055 [==============================] - 2s 636us/step - loss: 0.4871 - acc: 0.7804 - val_loss: 0.4833 - val_acc: 0.7801
Epoch 81/150
3055/3055 [==============================] - 2s 649us/step - loss: 0.4893 - acc: 0.7764 - val_loss: 0.4834 - val_acc: 0.7801
Epoch 82/150
3055/3055 [==============================] - 2s 657us/step - loss: 0.4884 - acc: 0.7820 - val_loss: 0.4833 - val_acc: 0.7801
Epoch 83/150
3055/3055 [==============================] - 2s 641us/step - loss: 0.4895 - acc: 0.7800 - val_loss: 0.4833 - val_acc: 0.7801
Epoch 84/150
3055/3055 [==============================] - 2s 634us/step - loss: 0.4896 - acc: 0.7810 - val_loss: 0.4834 - val_acc: 0.7801
Epoch 85/

Epoch 137/150
3055/3055 [==============================] - 2s 649us/step - loss: 0.4888 - acc: 0.7787 - val_loss: 0.4840 - val_acc: 0.7788
Epoch 138/150
3055/3055 [==============================] - 2s 643us/step - loss: 0.4880 - acc: 0.7794 - val_loss: 0.4838 - val_acc: 0.7788
Epoch 139/150
3055/3055 [==============================] - 2s 640us/step - loss: 0.4823 - acc: 0.7840 - val_loss: 0.4837 - val_acc: 0.7801
Epoch 140/150
3055/3055 [==============================] - 2s 649us/step - loss: 0.4875 - acc: 0.7791 - val_loss: 0.4836 - val_acc: 0.7788
Epoch 141/150
3055/3055 [==============================] - 2s 647us/step - loss: 0.4850 - acc: 0.7807 - val_loss: 0.4837 - val_acc: 0.7788
Epoch 142/150
3055/3055 [==============================] - 2s 655us/step - loss: 0.4889 - acc: 0.7810 - val_loss: 0.4837 - val_acc: 0.7788
Epoch 143/150
3055/3055 [==============================] - 2s 703us/step - loss: 0.4863 - acc: 0.7787 - val_loss: 0.4835 - val_acc: 0.7814
Epoch 144/150
3055/3055 [==

...

3055/3055 [==============================] - 2s 645us/step - loss: 0.4872 - acc: 0.7817 - val_loss: 0.4812 - val_acc: 0.7866
Epoch 191/200
3055/3055 [==============================] - 2s 685us/step - loss: 0.4843 - acc: 0.7774 - val_loss: 0.4810 - val_acc: 0.7866
Epoch 192/200
3055/3055 [==============================] - 2s 662us/step - loss: 0.4894 - acc: 0.7813 - val_loss: 0.4813 - val_acc: 0.7866
Epoch 193/200
3055/3055 [==============================] - 2s 650us/step - loss: 0.4853 - acc: 0.7846 - val_loss: 0.4814 - val_acc: 0.7853
Epoch 194/200
3055/3055 [==============================] - 2s 648us/step - loss: 0.4890 - acc: 0.7836 - val_loss: 0.4813 - val_acc: 0.7853
Epoch 195/200
3055/3055 [==============================] - 2s 650us/step - loss: 0.4816 - acc: 0.7804 - val_loss: 0.4810 - val_acc: 0.7866
Epoch 196/200
3055/3055 [==============================] - 2s 646us/step - loss: 0.4863 - acc: 0.7781 - val_loss: 0.4810 - val_acc: 0.7853
Epoch 197/200
3055/3055 [================

Epoch 7/200
3055/3055 [==============================] - 2s 631us/step - loss: 0.5091 - acc: 0.7650 - val_loss: 0.4853 - val_acc: 0.7788
Epoch 8/200
3055/3055 [==============================] - 2s 627us/step - loss: 0.5061 - acc: 0.7722 - val_loss: 0.4835 - val_acc: 0.7840
Epoch 9/200
3055/3055 [==============================] - 2s 639us/step - loss: 0.5029 - acc: 0.7758 - val_loss: 0.4849 - val_acc: 0.7840
Epoch 10/200
3055/3055 [==============================] - 2s 626us/step - loss: 0.4972 - acc: 0.7718 - val_loss: 0.4946 - val_acc: 0.7762
Epoch 11/200
3055/3055 [==============================] - 2s 632us/step - loss: 0.4975 - acc: 0.7777 - val_loss: 0.4962 - val_acc: 0.7696
Epoch 12/200
3055/3055 [==============================] - 2s 626us/step - loss: 0.4945 - acc: 0.7709 - val_loss: 0.4849 - val_acc: 0.7696
Epoch 13/200
3055/3055 [==============================] - 2s 640us/step - loss: 0.4940 - acc: 0.7797 - val_loss: 0.4865 - val_acc: 0.7801

Epoch 00013: ReduceLROnPlateau reduc

Epoch 65/200
3055/3055 [==============================] - 2s 681us/step - loss: 0.4843 - acc: 0.7823 - val_loss: 0.4849 - val_acc: 0.7827
Epoch 66/200
3055/3055 [==============================] - 2s 653us/step - loss: 0.4928 - acc: 0.7791 - val_loss: 0.4847 - val_acc: 0.7801
Epoch 67/200
3055/3055 [==============================] - 2s 633us/step - loss: 0.4877 - acc: 0.7794 - val_loss: 0.4847 - val_acc: 0.7801
Epoch 68/200
3055/3055 [==============================] - 2s 629us/step - loss: 0.4868 - acc: 0.7787 - val_loss: 0.4848 - val_acc: 0.7801
Epoch 69/200
3055/3055 [==============================] - 2s 630us/step - loss: 0.4873 - acc: 0.7820 - val_loss: 0.4849 - val_acc: 0.7827
Epoch 70/200
3055/3055 [==============================] - 2s 644us/step - loss: 0.4896 - acc: 0.7804 - val_loss: 0.4848 - val_acc: 0.7801
Epoch 71/200
3055/3055 [==============================] - 2s 628us/step - loss: 0.4922 - acc: 0.7784 - val_loss: 0.4847 - val_acc: 0.7801
Epoch 72/200
3055/3055 [==========

3055/3055 [==============================] - 2s 635us/step - loss: 0.4910 - acc: 0.7771 - val_loss: 0.4849 - val_acc: 0.7814
Epoch 184/200
3055/3055 [==============================] - 2s 630us/step - loss: 0.4907 - acc: 0.7830 - val_loss: 0.4849 - val_acc: 0.7814
Epoch 185/200
3055/3055 [==============================] - 2s 689us/step - loss: 0.4821 - acc: 0.7787 - val_loss: 0.4849 - val_acc: 0.7814
Epoch 186/200
3055/3055 [==============================] - 2s 640us/step - loss: 0.4897 - acc: 0.7764 - val_loss: 0.4849 - val_acc: 0.7814
Epoch 187/200
3055/3055 [==============================] - 2s 639us/step - loss: 0.4869 - acc: 0.7777 - val_loss: 0.4850 - val_acc: 0.7814
Epoch 188/200
3055/3055 [==============================] - 2s 637us/step - loss: 0.4849 - acc: 0.7823 - val_loss: 0.4850 - val_acc: 0.7814
Epoch 189/200
3055/3055 [==============================] - 2s 636us/step - loss: 0.4896 - acc: 0.7768 - val_loss: 0.4850 - val_acc: 0.7814
Epoch 190/200
3055/3055 [================

3055/3055 [==============================] - 2s 658us/step - loss: 0.4967 - acc: 0.7741 - val_loss: 0.4815 - val_acc: 0.7801
Epoch 42/50
3055/3055 [==============================] - 2s 649us/step - loss: 0.4932 - acc: 0.7774 - val_loss: 0.4816 - val_acc: 0.7801
Epoch 43/50
3055/3055 [==============================] - 2s 650us/step - loss: 0.4954 - acc: 0.7797 - val_loss: 0.4816 - val_acc: 0.7801
Epoch 44/50
3055/3055 [==============================] - 2s 751us/step - loss: 0.4897 - acc: 0.7768 - val_loss: 0.4818 - val_acc: 0.7814
Epoch 45/50
3055/3055 [==============================] - 2s 735us/step - loss: 0.4913 - acc: 0.7777 - val_loss: 0.4816 - val_acc: 0.7801
Epoch 46/50
3055/3055 [==============================] - 3s 903us/step - loss: 0.4944 - acc: 0.7774 - val_loss: 0.4817 - val_acc: 0.7814
Epoch 47/50
3055/3055 [==============================] - 2s 672us/step - loss: 0.4927 - acc: 0.7761 - val_loss: 0.4818 - val_acc: 0.7801
Epoch 48/50
3055/3055 [==============================

In [3]:
0.93484

0.93484